In [55]:
import pymongo
from pymongo import MongoClient
from pymongo import GEO2D, GEOSPHERE
from pymongo import GEO2D
from datetime import datetime
from pprint import pprint
from bson.objectid import ObjectId
import re
import string
import operator
from geojson import Point, Polygon, MultiPolygon, Feature, FeatureCollection
from math import log

from pymongo.mongo_client import MongoClient
import datetime
import json
import geojson

#if you don't have geo jason installed, use pip install geojson
#if you want to start mongodb use:  sudo service mongod start
#Monday is 0 -> week day

In [2]:
%%bash
service mongod status

mongod start/running, process 15470


In [3]:
client = MongoClient('mongodb://localhost:27017')
db = client.MyDB
db.collection_names()
#client.database_names()

['green_may', 'neighborhoods']

In [45]:
# Count by Borough-Pickups and dropoffs
def count_rides_by_boro(collection):
    group = {'$group': {'_id': '$pickup_b_code', 'count': {'$sum': 1}}}
    sort = {'$sort': {'_id': 1}}
    cursor = collection.aggregate([group,sort])
    pprint('Pickups')
    for c in cursor:
        print(c)
    group = {'$group': {'_id': '$dropoff_b_code', 'count': {'$sum': 1}}}
    sort = {'$sort': {'_id': 1}}
    cursor = collection.aggregate([group,sort])
    pprint('Dropoffs')
    for c in cursor:
        print(c)
count_rides_by_boro(collection)

'Pickups'
{'_id': None, 'count': 4206}
{'_id': 1, 'count': 439444}
{'_id': 2, 'count': 113493}
{'_id': 3, 'count': 318069}
{'_id': 4, 'count': 546185}
{'_id': 5, 'count': 106}
'Dropoffs'
{'_id': None, 'count': 7651}
{'_id': 1, 'count': 540987}
{'_id': 2, 'count': 130551}
{'_id': 3, 'count': 263831}
{'_id': 4, 'count': 478164}
{'_id': 5, 'count': 319}


In [44]:
# Count by Neighborhood - Pickups and Dorpoffs
def count_rides_by_neigh(collection):
    group = {'$group': {'_id': '$pickup_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'_id': 1}}
    cursor = collection.aggregate([group,sort])
    pprint('Pickups')
    for c in cursor:
        print(c)
    group = {'$group': {'_id': '$dropoff_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'_id': 1}}
    cursor = collection.aggregate([group,sort])
    pprint('Dropoffs')
    for c in cursor:
        print(c)
count_rides_by_neigh(collection)

'Pickups'
{'_id': None, 'count': 4206}
{'_id': 'Allerton', 'count': 1218}
{'_id': 'Alley Pond Park', 'count': 8}
{'_id': 'Arden Heights', 'count': 2}
{'_id': 'Arrochar', 'count': 3}
{'_id': 'Arverne', 'count': 8}
{'_id': 'Astoria', 'count': 77170}
{'_id': 'Bath Beach', 'count': 64}
{'_id': 'Battery Park City', 'count': 6}
{'_id': 'Bay Ridge', 'count': 681}
{'_id': 'Bay Terrace', 'count': 27}
{'_id': 'Baychester', 'count': 235}
{'_id': 'Bayside', 'count': 330}
{'_id': 'Bayswater', 'count': 7}
{'_id': 'Bedford-Stuyvesant', 'count': 32087}
{'_id': 'Belle Harbor', 'count': 2}
{'_id': 'Bellerose', 'count': 52}
{'_id': 'Belmont', 'count': 3571}
{'_id': 'Bensonhurst', 'count': 137}
{'_id': 'Bergen Beach', 'count': 37}
{'_id': 'Boerum Hill', 'count': 16288}
{'_id': 'Borough Park', 'count': 541}
{'_id': 'Breezy Point', 'count': 3}
{'_id': 'Briarwood', 'count': 946}
{'_id': 'Brighton Beach', 'count': 454}
{'_id': 'Broad Channel', 'count': 2}
{'_id': 'Bronx Park', 'count': 565}
{'_id': 'Bronxdale

'Dropoffs'
{'_id': None, 'count': 7651}
{'_id': 'Allerton', 'count': 1995}
{'_id': 'Alley Pond Park', 'count': 13}
{'_id': 'Arden Heights', 'count': 6}
{'_id': 'Arrochar', 'count': 9}
{'_id': 'Arverne', 'count': 28}
{'_id': 'Astoria', 'count': 49025}
{'_id': 'Bath Beach', 'count': 314}
{'_id': 'Battery Park City', 'count': 2032}
{'_id': 'Bay Ridge', 'count': 1897}
{'_id': 'Bay Terrace', 'count': 286}
{'_id': 'Baychester', 'count': 418}
{'_id': 'Bayside', 'count': 1408}
{'_id': 'Bayswater', 'count': 37}
{'_id': 'Bedford-Stuyvesant', 'count': 37589}
{'_id': 'Belle Harbor', 'count': 37}
{'_id': 'Bellerose', 'count': 276}
{'_id': 'Belmont', 'count': 4248}
{'_id': 'Bensonhurst', 'count': 559}
{'_id': 'Bergen Beach', 'count': 149}
{'_id': 'Bloomfield', 'count': 2}
{'_id': 'Boerum Hill', 'count': 7207}
{'_id': 'Borough Park', 'count': 1551}
{'_id': 'Breezy Point', 'count': 28}
{'_id': 'Briarwood', 'count': 3621}
{'_id': 'Brighton Beach', 'count': 796}
{'_id': 'Broad Channel', 'count': 17}
{'_

In [43]:
# pickups and dropoffs in Neighborhoods of specific Borough
def rides_in_neigh_of_boro(collection, boro):
    if boro not in range(1,6):
        pprint('Enter a Borocode from 1 to 5')
        return
    match = {'$match': {'pickup_b_code': boro}}
    group = {'$group': {'_id': '$pickup_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}
    cursor = collection.aggregate([match, group,sort])
    pprint('Pickups')
    for c in cursor:
        print(c)
        
    match = {'$match': {'dropoff_b_code': boro}}
    group = {'$group': {'_id': '$dropoff_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}
    cursor = collection.aggregate([match, group,sort])
    pprint('Pickups')
    for c in cursor:
        print(c)
rides_in_neigh_of_boro(collection, 3)

'Pickups'
{'_id': 'Bedford-Stuyvesant', 'count': 32087}
{'_id': 'Greenpoint', 'count': 31873}
{'_id': 'Park Slope', 'count': 29945}
{'_id': 'Fort Greene', 'count': 27380}
{'_id': 'Downtown Brooklyn', 'count': 26908}
{'_id': 'Brooklyn Heights', 'count': 20286}
{'_id': 'DUMBO', 'count': 17340}
{'_id': 'Crown Heights', 'count': 17132}
{'_id': 'Boerum Hill', 'count': 16288}
{'_id': 'Clinton Hill', 'count': 15067}
{'_id': 'Carroll Gardens', 'count': 14333}
{'_id': 'Cobble Hill', 'count': 9833}
{'_id': 'Prospect Heights', 'count': 8534}
{'_id': 'Gowanus', 'count': 7214}
{'_id': 'South Slope', 'count': 6874}
{'_id': 'Sheepshead Bay', 'count': 5975}
{'_id': 'Flatbush', 'count': 4855}
{'_id': 'Sunset Park', 'count': 4447}
{'_id': 'Prospect-Lefferts Gardens', 'count': 3591}
{'_id': 'Prospect Park', 'count': 2282}
{'_id': 'East Flatbush', 'count': 2144}
{'_id': 'Red Hook', 'count': 1860}
{'_id': 'Vinegar Hill', 'count': 1696}
{'_id': 'Windsor Terrace', 'count': 1215}
{'_id': 'Kensington', 'count'

In [51]:
#Borough to borough traffic
def boro_to_boro(collection):
    project = {'$project': {'tofrom':{'$add': [{'$multiply':['$pickup_b_code',10]},'$dropoff_b_code']}}}
    group = {'$group': {'_id': '$tofrom', 'count': {'$sum': 1}}}
    sort = {'$sort': {'_id': 1}}
    cursor = collection.aggregate([project, group, sort])
    for c in cursor:
        pprint(c)
boro_to_boro(collection)

{'_id': 11, 'count': 376120}
{'_id': None, 'count': 10357}
{'_id': 11, 'count': 376120}
{'_id': 12, 'count': 42284}
{'_id': 13, 'count': 2760}
{'_id': 14, 'count': 16257}
{'_id': 15, 'count': 20}
{'_id': 21, 'count': 24871}
{'_id': 22, 'count': 85151}
{'_id': 23, 'count': 332}
{'_id': 24, 'count': 1762}
{'_id': 25, 'count': 2}
{'_id': 31, 'count': 63182}
{'_id': 32, 'count': 349}
{'_id': 33, 'count': 197419}
{'_id': 34, 'count': 56114}
{'_id': 35, 'count': 144}
{'_id': 41, 'count': 75951}
{'_id': 42, 'count': 2374}
{'_id': 43, 'count': 62741}
{'_id': 44, 'count': 403132}
{'_id': 45, 'count': 79}
{'_id': 51, 'count': 13}
{'_id': 52, 'count': 1}
{'_id': 53, 'count': 12}
{'_id': 54, 'count': 6}
{'_id': 55, 'count': 70}


In [67]:
# neighbourhood to neighbourhood traffic
def neigh_to_neigh(collection)
    project = {'$project': {'tofrom':{'$concat': ['$pickup_n_Name','_to_','$dropoff_n_Name']}}}
    group = {'$group': {'_id': '$tofrom', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}
    cursor = collection.aggregate([project, group, sort])
    for c in cursor:
        pprint(c)
neigh_to_neigh(collection)

{'_id': 'Harlem_to_Harlem', 'count': 53980}
{'_id': 'Williamsburg_to_Williamsburg', 'count': 29444}
{'_id': 'East Harlem_to_East Harlem', 'count': 28225}
{'_id': 'Astoria_to_Astoria', 'count': 22570}
{'_id': 'East Harlem_to_Harlem', 'count': 22282}
{'_id': 'East Harlem_to_Upper East Side', 'count': 19863}
{'_id': 'Harlem_to_East Harlem', 'count': 19318}
{'_id': 'Morningside Heights_to_Upper West Side', 'count': 16170}
{'_id': 'Harlem_to_Upper West Side', 'count': 14609}
{'_id': 'Williamsburg_to_Greenpoint', 'count': 14070}
{'_id': 'Washington Heights_to_Washington Heights', 'count': 12965}
{'_id': 'East Harlem_to_Upper West Side', 'count': 12804}
{'_id': 'Morningside Heights_to_Harlem', 'count': 11048}
{'_id': None, 'count': 10357}
{'_id': 'Harlem_to_Washington Heights', 'count': 9935}
{'_id': 'Long Island City_to_Long Island City', 'count': 9914}
{'_id': 'Elmhurst_to_Elmhurst', 'count': 9824}
{'_id': 'Greenpoint_to_Williamsburg', 'count': 9107}
{'_id': 'Astoria_to_Long Island City', '

{'_id': 'Concourse Village_to_Morrisania', 'count': 242}
{'_id': 'Ditmars Steinway_to_Elmhurst', 'count': 242}
{'_id': 'Jackson Heights_to_College Point', 'count': 242}
{'_id': 'Flatbush_to_Bedford-Stuyvesant', 'count': 241}
{'_id': 'East Harlem_to_Greenwich Village', 'count': 241}
{'_id': 'Carroll Gardens_to_Clinton Hill', 'count': 241}
{'_id': 'Clinton Hill_to_Lower East Side', 'count': 241}
{'_id': 'Brooklyn Heights_to_Sunset Park', 'count': 241}
{'_id': 'Washington Heights_to_Murray Hill', 'count': 240}
{'_id': 'Woodside_to_Ridgewood', 'count': 240}
{'_id': 'Boerum Hill_to_Cobble Hill', 'count': 239}
{'_id': 'Fort Greene_to_John F. Kennedy International Airport', 'count': 239}
{'_id': 'Downtown Brooklyn_to_West Village', 'count': 239}
{'_id': 'South Slope_to_Gowanus', 'count': 239}
{'_id': 'Sunset Park_to_Williamsburg', 'count': 239}
{'_id': 'Boerum Hill_to_Chelsea', 'count': 238}
{'_id': 'Richmond Hill_to_Jamaica', 'count': 238}
{'_id': 'Forest Hills_to_Woodside', 'count': 238}
{'

{'_id': 'Flushing Meadows Corona Park_to_Upper West Side', 'count': 55}
{'_id': 'Vinegar Hill_to_Boerum Hill', 'count': 55}
{'_id': 'Prospect Heights_to_Financial District', 'count': 55}
{'_id': 'Red Hook_to_Crown Heights', 'count': 55}
{'_id': 'Morningside Heights_to_Claremont Village', 'count': 55}
{'_id': 'Navy Yard_to_Downtown Brooklyn', 'count': 55}
{'_id': 'East Elmhurst_to_John F. Kennedy International Airport', 'count': 55}
{'_id': 'Woodside_to_Central Park', 'count': 55}
{'_id': 'Allerton_to_Kingsbridge', 'count': 55}
{'_id': 'Washington Heights_to_Ditmars Steinway', 'count': 54}
{'_id': 'Bushwick_to_Brownsville', 'count': 54}
{'_id': 'Astoria_to_Chinatown', 'count': 54}
{'_id': 'Kew Gardens_to_Upper East Side', 'count': 54}
{'_id': 'Williamsburg_to_Inwood', 'count': 54}
{'_id': 'Sheepshead Bay_to_Mill Basin', 'count': 54}
{'_id': 'Gowanus_to_Columbia St', 'count': 54}
{'_id': 'Soundview_to_Fordham', 'count': 54}
{'_id': 'Unionport_to_Harlem', 'count': 54}
{'_id': 'Sunset Park

{'_id': 'East Harlem_to_Sunset Park', 'count': 22}
{'_id': 'Ditmars Steinway_to_Kew Gardens', 'count': 22}
{'_id': 'Bushwick_to_East Harlem', 'count': 22}
{'_id': 'University Heights_to_Marble Hill', 'count': 22}
{'_id': 'Mount Hope_to_Allerton', 'count': 22}
{'_id': 'Jackson Heights_to_East New York', 'count': 22}
{'_id': 'DUMBO_to_Two Bridges', 'count': 22}
{'_id': 'Fordham_to_Crotona Park', 'count': 22}
{'_id': 'East Harlem_to_Spuyten Duyvil', 'count': 22}
{'_id': 'Elmhurst_to_Central Park', 'count': 22}
{'_id': 'East Harlem_to_Borough Park', 'count': 22}
{'_id': 'Allerton_to_Van Nest', 'count': 22}
{'_id': 'Morris Heights_to_West Farms', 'count': 22}
{'_id': 'Sunset Park_to_Dyker Heights', 'count': 22}
{'_id': 'Park Slope_to_Washington Heights', 'count': 22}
{'_id': 'Morrisania_to_Inwood', 'count': 22}
{'_id': 'Crotona Park_to_Crotona Park', 'count': 22}
{'_id': 'Bronxdale_to_Mott Haven', 'count': 22}
{'_id': 'Sunnyside_to_Nolita', 'count': 22}
{'_id': 'Corona_to_Woodhaven', 'count

{'_id': 'Ditmars Steinway_to_Westchester Square', 'count': 10}
{'_id': 'Pelham Gardens_to_Bronxdale', 'count': 10}
{'_id': 'Brooklyn Heights_to_Canarsie', 'count': 10}
{'_id': 'Forest Hills_to_Brownsville', 'count': 10}
{'_id': 'Vinegar Hill_to_Kips Bay', 'count': 10}
{'_id': 'Brooklyn Heights_to_Flushing', 'count': 10}
{'_id': 'Rego Park_to_Roosevelt Island', 'count': 10}
{'_id': 'Carroll Gardens_to_Marine Park', 'count': 10}
{'_id': 'Tremont_to_Bronx Park', 'count': 10}
{'_id': 'Kensington_to_SoHo', 'count': 10}
{'_id': 'Schuylerville_to_Allerton', 'count': 10}
{'_id': 'Longwood_to_Long Island City', 'count': 10}
{'_id': 'Cypress Hills_to_Canarsie', 'count': 10}
{'_id': 'Richmond Hill_to_Astoria', 'count': 10}
{'_id': 'Marble Hill_to_Mott Haven', 'count': 10}
{'_id': 'Morrisania_to_Van Nest', 'count': 10}
{'_id': 'Marble Hill_to_Highbridge', 'count': 10}
{'_id': 'Cypress Hills_to_Richmond Hill', 'count': 10}
{'_id': 'Throgs Neck_to_Allerton', 'count': 10}
{'_id': 'Bushwick_to_Midwood

...

{'_id': 'Sunnyside_to_Williamsbridge', 'count': 2}
{'_id': 'John F. Kennedy International Airport_to_Parkchester', 'count': 2}
{'_id': 'College Point_to_Borough Park', 'count': 2}
{'_id': 'Jamaica_to_Allerton', 'count': 2}
{'_id': 'Carroll Gardens_to_Maspeth', 'count': 2}
{'_id': 'Flushing Meadows Corona Park_to_Longwood', 'count': 2}
{'_id': 'Fieldston_to_Van Cortlandt Park', 'count': 2}
{'_id': 'Ditmars Steinway_to_Howard Beach', 'count': 2}
{'_id': 'Downtown Brooklyn_to_Throgs Neck', 'count': 2}
{'_id': 'Breezy Point_to_Rockaway Beach', 'count': 2}
{'_id': 'Prospect Heights_to_Bath Beach', 'count': 2}
{'_id': 'Navy Yard_to_Kensington', 'count': 2}
{'_id': 'Flushing_to_Pelham Bay', 'count': 2}
{'_id': 'Bronxdale_to_Riverdale', 'count': 2}
{'_id': 'Sheepshead Bay_to_East Elmhurst', 'count': 2}
{'_id': 'Van Nest_to_Crown Heights', 'count': 2}
{'_id': 'East New York_to_West Village', 'count': 2}
{'_id': 'St. Albans_to_Richmond Hill', 'count': 2}
{'_id': 'John F. Kennedy International Ai

{'_id': 'Gravesend_to_Windsor Terrace', 'count': 2}
{'_id': "Randall's Island_to_Clinton Hill", 'count': 2}
{'_id': 'Hunts Point_to_Midtown', 'count': 2}
{'_id': 'Bronx Park_to_LaGuardia Airport', 'count': 2}
{'_id': 'Bronxdale_to_Morrisania', 'count': 2}
{'_id': 'Red Hook_to_Bensonhurst', 'count': 2}
{'_id': 'Edgemere_to_Edgemere', 'count': 2}
{'_id': 'Central Park_to_Kingsbridge', 'count': 2}
{'_id': 'Cobble Hill_to_College Point', 'count': 2}
{'_id': 'Edenwald_to_Mott Haven', 'count': 2}
{'_id': 'Bay Ridge_to_Bushwick', 'count': 2}
{'_id': 'Melrose_to_Woodhaven', 'count': 2}
{'_id': 'Marble Hill_to_Throgs Neck', 'count': 2}
{'_id': 'Schuylerville_to_Melrose', 'count': 2}
{'_id': 'Flushing_to_Parkchester', 'count': 2}
{'_id': 'East New York_to_Borough Park', 'count': 2}
{'_id': 'Whitestone_to_LaGuardia Airport', 'count': 2}
{'_id': 'Bronx Park_to_Hunts Point', 'count': 2}
{'_id': 'Downtown Brooklyn_to_Randall Manor', 'count': 2}
{'_id': 'East New York_to_Astoria', 'count': 2}
{'_id':

{'_id': 'Port Morris_to_Bayside', 'count': 1}
{'_id': 'John F. Kennedy International Airport_to_North Riverdale', 'count': 1}
{'_id': 'Jamaica Estates_to_Murray Hill', 'count': 1}
{'_id': 'Bedford-Stuyvesant_to_South Beach', 'count': 1}
{'_id': 'Kew Gardens Hills_to_Chinatown', 'count': 1}
{'_id': 'Riverdale_to_Pelham Gardens', 'count': 1}
{'_id': 'Spuyten Duyvil_to_Bedford-Stuyvesant', 'count': 1}
{'_id': 'Green-Wood Cemetery_to_Bensonhurst', 'count': 1}
{'_id': 'Park Slope_to_College Point', 'count': 1}
{'_id': 'LaGuardia Airport_to_College Point', 'count': 1}
{'_id': 'Flatbush_to_Little Italy', 'count': 1}
{'_id': 'Fort Hamilton_to_Long Island City', 'count': 1}
{'_id': 'Crotona Park_to_LaGuardia Airport', 'count': 1}
{'_id': 'Allerton_to_Morningside Heights', 'count': 1}
{'_id': 'Clinton Hill_to_Allerton', 'count': 1}
{'_id': 'Upper West Side_to_Mount Eden', 'count': 1}
{'_id': 'Jamaica Hills_to_Financial District', 'count': 1}
{'_id': 'Howard Beach_to_Lower East Side', 'count': 1}

{'_id': 'Morris Park_to_Financial District', 'count': 1}
{'_id': 'Rockaway Park_to_East New York', 'count': 1}
{'_id': 'New Springville_to_Port Richmond', 'count': 1}
{'_id': 'Co-op City_to_College Point', 'count': 1}
{'_id': 'East New York_to_Rockaway Park', 'count': 1}
{'_id': 'Co-op City_to_Bronx Park', 'count': 1}
{'_id': 'Green-Wood Cemetery_to_Columbia St', 'count': 1}
{'_id': "Randall's Island_to_Bellerose", 'count': 1}
{'_id': 'Park Slope_to_New Springville', 'count': 1}
{'_id': 'East Elmhurst_to_North Riverdale', 'count': 1}
{'_id': 'Briarwood_to_Downtown Brooklyn', 'count': 1}
{'_id': 'Financial District_to_East Flatbush', 'count': 1}
{'_id': 'Sheepshead Bay_to_Jamaica Estates', 'count': 1}
{'_id': 'Jamaica Hills_to_Bedford-Stuyvesant', 'count': 1}
{'_id': 'Prospect-Lefferts Gardens_to_Co-op City', 'count': 1}
{'_id': 'Long Island City_to_Williamsbridge', 'count': 1}
{'_id': 'Richmond Hill_to_Midwood', 'count': 1}
{'_id': 'Belle Harbor_to_Brighton Beach', 'count': 1}
{'_id': 

{'_id': 'Morris Park_to_Marble Hill', 'count': 1}
{'_id': 'Red Hook_to_Jamaica', 'count': 1}
{'_id': 'Morris Heights_to_Gramercy', 'count': 1}
{'_id': 'Roosevelt Island_to_Fort Hamilton', 'count': 1}
{'_id': 'Downtown Brooklyn_to_Bronx Park', 'count': 1}
{'_id': 'Concourse Village_to_Marine Park', 'count': 1}
{'_id': 'Arrochar_to_St. George', 'count': 1}
{'_id': 'East New York_to_Fordham', 'count': 1}
{'_id': 'Williamsbridge_to_Throgs Neck', 'count': 1}
{'_id': 'Hunts Point_to_SoHo', 'count': 1}
{'_id': 'Greenpoint_to_Concord', 'count': 1}
{'_id': 'Bedford-Stuyvesant_to_Rockaway Beach', 'count': 1}
{'_id': 'Allerton_to_Prospect Heights', 'count': 1}
{'_id': 'Sunnyside_to_Bath Beach', 'count': 1}
{'_id': 'Bayside_to_Elmhurst', 'count': 1}
{'_id': 'Maspeth_to_Holliswood', 'count': 1}
{'_id': 'Midwood_to_Rosedale', 'count': 1}
{'_id': 'Harlem_to_Park Hill', 'count': 1}
{'_id': 'Briarwood_to_Nolita', 'count': 1}
{'_id': 'Fort Hamilton_to_Lower East Side', 'count': 1}
{'_id': 'Sunnyside_to_

In [59]:
#Distribution of trip distance for A to B boroughs
def distance_distribution_boro_boro(collection, boroA, boroB):
    if boroA not in range(1,6):
        pprint('Enter a Borocode from 1 to 5')
        return
    if boroA not in range(1,6):
        pprint('Enter a Borocode from 1 to 5')
        return
    AB = 10*boroA + boroB
    pprint(AB)
    filter0 = {'$match': {'Trip_distance': {'$gt' : 0.0}}} 
    project = {'$project': {'tofrom':{'$add': [{'$multiply':['$pickup_b_code',10]},\
                                               '$dropoff_b_code']}, 'tdfloor':{'$floor': ['$Trip_distance']}}}# add filter to ignore 0 distance please
    match = {'$match': {'tofrom':AB}}
    group = {'$group': {'_id': '$tdfloor', 'count': {'$sum': 1}}}
    sort = {'$sort': {'_id': 1}}
    cursor = collection.aggregate([filter0,project, match, group, sort])
#     cursor.count()
    for c in cursor:
        print(c)
distance_distribution_boro_boro(collection, 1, 1)    

11
{'_id': 0.0, 'count': 93640}
{'_id': 1.0, 'count': 126169}
{'_id': 2.0, 'count': 58024}
{'_id': 3.0, 'count': 31141}
{'_id': 4.0, 'count': 19110}
{'_id': 5.0, 'count': 14506}
{'_id': 6.0, 'count': 10850}
{'_id': 7.0, 'count': 8067}
{'_id': 8.0, 'count': 4970}
{'_id': 9.0, 'count': 2822}
{'_id': 10.0, 'count': 1402}
{'_id': 11.0, 'count': 673}
{'_id': 12.0, 'count': 327}
{'_id': 13.0, 'count': 127}
{'_id': 14.0, 'count': 65}
{'_id': 15.0, 'count': 35}
{'_id': 16.0, 'count': 25}
{'_id': 17.0, 'count': 20}
{'_id': 18.0, 'count': 12}
{'_id': 19.0, 'count': 7}
{'_id': 20.0, 'count': 5}
{'_id': 21.0, 'count': 14}
{'_id': 22.0, 'count': 1}
{'_id': 23.0, 'count': 4}
{'_id': 24.0, 'count': 3}
{'_id': 25.0, 'count': 3}
{'_id': 26.0, 'count': 1}
{'_id': 28.0, 'count': 2}
{'_id': 29.0, 'count': 1}
{'_id': 30.0, 'count': 2}
{'_id': 31.0, 'count': 2}
{'_id': 33.0, 'count': 2}
{'_id': 35.0, 'count': 1}
{'_id': 36.0, 'count': 1}
{'_id': 37.0, 'count': 1}
{'_id': 39.0, 'count': 1}
{'_id': 40.0, 'cou

In [42]:
#Distribution of hour of pickup time for A to B boroughs pickups and dropoffs
def time_distribution_boro_boro(collection, boroA, boroB):
    if boroA not in range(1,6):
        pprint('Enter a Borocode from 1 to 5')
        return
    if boroA not in range(1,6):
        pprint('Enter a Borocode from 1 to 5')
        return
    AB = 10*boroA + boroB
    pprint(AB)
    project = {'$project': {'tofrom':{'$add': [{'$multiply':['$pickup_b_code',10]},\
                                               '$dropoff_b_code']}, 'hours': { '$hour': "$lpep_pickup_datetime" }}}
    match = {'$match': {'tofrom':AB}}
    group = {'$group': {'_id': '$hours', 'count': {'$sum': 1}}}
    sort = {'$sort': {'_id': 1}}
    cursor = collection.aggregate([project, match, group, sort])
    pprint('Pickup')
    for c in cursor:
        print(c)
    project = {'$project': {'tofrom':{'$add': [{'$multiply':['$pickup_b_code',10]},\
                                               '$dropoff_b_code']}, 'hours': { '$hour': "$Lpep_dropoff_datetime" }}}
    match = {'$match': {'tofrom':AB}}
    group = {'$group': {'_id': '$hours', 'count': {'$sum': 1}}}
    sort = {'$sort': {'_id': 1}}
    cursor = collection.aggregate([project, match, group, sort])
    pprint('Dropoff')
    for c in cursor:
        print(c)
time_distribution_boro_boro(collection, 1, 1)    
    



'Pickup'
{'_id': 0, 'count': 9950}
{'_id': 1, 'count': 7155}
{'_id': 2, 'count': 4827}
{'_id': 3, 'count': 3405}
{'_id': 4, 'count': 2741}
{'_id': 5, 'count': 2938}
{'_id': 6, 'count': 5746}
{'_id': 7, 'count': 13681}
{'_id': 8, 'count': 20381}
{'_id': 9, 'count': 20851}
{'_id': 10, 'count': 18755}
{'_id': 11, 'count': 17874}
{'_id': 12, 'count': 17739}
{'_id': 13, 'count': 18029}
{'_id': 14, 'count': 20208}
{'_id': 15, 'count': 22284}
{'_id': 16, 'count': 24423}
{'_id': 17, 'count': 25413}
{'_id': 18, 'count': 25988}
{'_id': 19, 'count': 24783}
{'_id': 20, 'count': 20699}
{'_id': 21, 'count': 18644}
{'_id': 22, 'count': 16149}
{'_id': 23, 'count': 13457}
'Dropoff'
{'_id': 0, 'count': 10670}
{'_id': 1, 'count': 7527}
{'_id': 2, 'count': 5133}
{'_id': 3, 'count': 3560}
{'_id': 4, 'count': 2808}
{'_id': 5, 'count': 2759}
{'_id': 6, 'count': 5049}
{'_id': 7, 'count': 11551}
{'_id': 8, 'count': 19035}
{'_id': 9, 'count': 21403}
{'_id': 10, 'count': 19098}
{'_id': 11, 'count': 17923}
{'_id'

In [47]:
#Creates a geoJson file, shows the percent of night pickups compared to all day per neighborhood
#use geojson.io to view the geojson file
def pickup_nightlife(collection, Jsonfilename):
    group = {'$group': {'_id': '$pickup_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}

    cursor = collection.aggregate([group,sort])
    p_neighborhoodTotals = dict()
    for c in cursor:
        p_neighborhoodTotals[c['_id']] = c['count']
    pprint('Calculated Totals')    
    project = {'$project' : {'pickup_hour': {'$hour': "$lpep_pickup_datetime"}, 'pickup_n_Name':1}}
    match = {'$match': {'pickup_hour': {'$lte' : 5}}} 
    group = {'$group': {'_id': '$pickup_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}

    cursor = collection.aggregate([project,match,group, sort])

    p_neighborhoodNightTotals = dict()
    for c in cursor:
        p_neighborhoodNightTotals[c['_id']] = c['count']
    pprint('Calculated Night Totals')    
    p_neighborhoodNightPercent = dict()
    for key,value in p_neighborhoodTotals.items():
        if key in p_neighborhoodNightTotals:
            p_neighborhoodNightPercent[key] =  p_neighborhoodNightTotals[key]/value
        else:
            p_neighborhoodNightPercent[key] =  0
    sorted_p_neighborhoodNightPercent = sorted(p_neighborhoodNightPercent.items(), key=operator.itemgetter(1), reverse = True)
    pprint('Calculated Night Percent')
    for s in sorted_p_neighborhoodNightPercent:
        pprint(s[0])
        pprint(s[1])
    features = []
    c = db['neighborhoods'].find()
    for n in c:
        coordinates = []
        coordinates.append(n['coordinates'])
        if n['neighborhood'] in p_neighborhoodNightPercent:
            percent = p_neighborhoodNightPercent[n['neighborhood']]
        else:
            percent = 0
        features.append(Feature(geometry=Polygon(coordinates), properties={"stroke": "#555555", "stroke-width": 2, "stroke-opacity": 1, "fill": "#25208a", "fill-opacity": percent}))
    feature_collection = FeatureCollection(features)
    with open(Jsonfilename, 'w') as outfile:
        geojson.dump(feature_collection, outfile)
pickup_nightlife(collection, 'pickup_nightlife.geojson')

'Calculated Totals'
'Calculated Night Totals'
'Calculated Night Percent'
'Arden Heights'
1.0
'Oakwood'
1.0
'Woodrow'
1.0
'Randall Manor'
1.0
'West Brighton'
1.0
'Dongan Hills'
1.0
'Gerritsen Beach'
0.5
'Arverne'
0.5
'Emerson Hill'
0.5
'Rockaway Beach'
0.5
'Little Italy'
0.5
'Broad Channel'
0.5
'South Beach'
0.5
'Bayside'
0.46060606060606063
'Laurelton'
0.42857142857142855
'Two Bridges'
0.4
'Bushwick'
0.38629885057471264
None
0.38492629576795057
'Chinatown'
0.375
'Eastchester'
0.35384615384615387
'Williamsburg'
0.34835791666019056
'Douglaston'
0.3333333333333333
'Rockaway Park'
0.3333333333333333
'Neponsit'
0.3333333333333333
'Arrochar'
0.3333333333333333
'Jackson Heights'
0.30024574760275624
'South Ozone Park'
0.2857142857142857
'Ridgewood'
0.28159645232815966
'Whitestone'
0.2711864406779661
'West Village'
0.26666666666666666
'East Village'
0.2631578947368421
'Hunts Point'
0.2603036876355748
'Flatlands'
0.25956284153005466
'Lower East Side'
0.2571428571428571
'Alley Pond Park'
0.25
'Ed

In [46]:
#Creates a geoJson file, shows the percent of night dropoffs compared to all day per neighborhood
#use geojson.io to view the geojson file
def dropoff_nightlife(collection, Jsonfilename):
    group = {'$group': {'_id': '$dropoff_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}

    cursor = collection.aggregate([group,sort])
    neighborhoodTotals = dict()
    for c in cursor:
        neighborhoodTotals[c['_id']] = c['count']
    pprint('Calculated Totals')    
    project = {'$project' : {'dropoff_hour': {'$hour': "$Lpep_dropoff_datetime"}, 'dropoff_n_Name':1}}
    match = {'$match': {'dropoff_hour': {'$lte' : 5}}} 
    group = {'$group': {'_id': '$dropoff_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}

    cursor = collection.aggregate([project,match,group, sort])

    neighborhoodNightTotals = dict()
    for c in cursor:
        neighborhoodNightTotals[c['_id']] = c['count']
    pprint('Calculated Night Totals')    
    neighborhoodNightPercent = dict()
    for key,value in neighborhoodTotals.items():
        if key in neighborhoodNightTotals:
            neighborhoodNightPercent[key] =  neighborhoodNightTotals[key]/value
        else:
            neighborhoodNightPercent[key] =  0
    sorted_neighborhoodNightPercent = sorted(neighborhoodNightPercent.items(), key=operator.itemgetter(1), reverse = True)
    pprint('Calculated Night Percent')
    for s in sorted_neighborhoodNightPercent:
        pprint(s[0])
        pprint(s[1])
    features = []
    c = db['neighborhoods'].find()
    for n in c:
        coordinates = []
        coordinates.append(n['coordinates'])
        if n['neighborhood'] in neighborhoodNightPercent:
            percent = neighborhoodNightPercent[n['neighborhood']]
        else:
            percent = 0
        features.append(Feature(geometry=Polygon(coordinates), properties={"stroke": "#555555", "stroke-width": 2, "stroke-opacity": 1, "fill": "#25208a", "fill-opacity": percent}))
    feature_collection = FeatureCollection(features)
    with open(Jsonfilename, 'w') as outfile:
        geojson.dump(feature_collection, outfile)
dropoff_nightlife(collection, 'dropoff_nightlife.geojson')

'Calculated Totals'
'Calculated Night Totals'
'Calculated Night Percent'
'Arden Heights'
1.0
'Willowbrook'
1.0
'Dongan Hills'
0.75
'Park Hill'
0.5
'Emerson Hill'
0.5
'Woodrow'
0.5
'Bloomfield'
0.5
'Silver Lake'
0.5
'Broad Channel'
0.47058823529411764
'Bushwick'
0.424571013358672
'Concord'
0.4166666666666667
'Rockaway Park'
0.3902439024390244
'Rockaway Beach'
0.3829787234042553
'Ridgewood'
0.37885361954784197
"Bull's Head"
0.375
'Douglaston'
0.3629032258064516
'Hollis Hills'
0.3383458646616541
'Arrochar'
0.3333333333333333
'Todt Hill'
0.3333333333333333
'Eltingville'
0.3333333333333333
'Oakwood'
0.3333333333333333
'Great Kills'
0.3333333333333333
'Flatlands'
0.3266832917705736
'Whitestone'
0.32408102025506375
'Randall Manor'
0.3181818181818182
'Bay Ridge'
0.30680021085925147
'Graniteville'
0.3
'Cambria Heights'
0.2978723404255319
'Bedford-Stuyvesant'
0.29750724946127854
'Bayswater'
0.2972972972972973
'Corona'
0.29360923601034633
'Fort Hamilton'
0.29205366357069146
'Jackson Heights'
0.28

In [57]:
#Creates a geoJson file, shows total count of pickups per neighborhood
#use geojson.io to view the geojson file
def pickup_by_neighborhood(collection, Jsonfilename):
    group = {'$group': {'_id': '$pickup_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}
    
    cursor = collection.aggregate([group,sort])
    p_neighborhoodTotals = dict()
    for c in cursor:
        p_neighborhoodTotals[c['_id']] = log(c['count'])
    pprint('Calculated Totals')    
    sorted_p_neighborhoodTotals = sorted(p_neighborhoodTotals.items(), key=operator.itemgetter(1), reverse = True)
    maximum_total = sorted_p_neighborhoodTotals[0][1]
    pprint(maximum_total)
    features = []
    c = db['neighborhoods'].find()
    for n in c:
        coordinates = []
        coordinates.append(n['coordinates'])
        if n['neighborhood'] in p_neighborhoodTotals:
            percent = p_neighborhoodTotals[n['neighborhood']]/maximum_total
        else:
            percent = 0
        features.append(Feature(geometry=Polygon(coordinates), properties={"stroke": "#555555", "stroke-width": 2, "stroke-opacity": 1, "fill": "#25208a", "fill-opacity": percent}))
    feature_collection = FeatureCollection(features)
    with open(Jsonfilename, 'w') as outfile:
        geojson.dump(feature_collection, outfile)
pickup_by_neighborhood(collection, 'pickup_by_neighborhood.geojson')

'Calculated Totals'
11.955223823123742


In [58]:
#Creates a geoJson file, shows total count of dropoffs per neighborhood
#use geojson.io to view the geojson file
def dropoff_by_neighborhood(collection, Jsonfilename):
    group = {'$group': {'_id': '$dropoff_n_Name', 'count': {'$sum': 1}}}
    sort = {'$sort': {'count': -1}}
    
    cursor = collection.aggregate([group,sort])
    p_neighborhoodTotals = dict()
    for c in cursor:
        p_neighborhoodTotals[c['_id']] = log(c['count'])
    pprint('Calculated Totals')    
    sorted_p_neighborhoodTotals = sorted(p_neighborhoodTotals.items(), key=operator.itemgetter(1), reverse = True)
    maximum_total = sorted_p_neighborhoodTotals[0][1]
    pprint(maximum_total)
    features = []
    c = db['neighborhoods'].find()
    for n in c:
        coordinates = []
        coordinates.append(n['coordinates'])
        if n['neighborhood'] in p_neighborhoodTotals:
            percent = p_neighborhoodTotals[n['neighborhood']]/maximum_total
        else:
            percent = 0
        features.append(Feature(geometry=Polygon(coordinates), properties={"stroke": "#555555", "stroke-width": 2, "stroke-opacity": 1, "fill": "#25208a", "fill-opacity": percent}))
    feature_collection = FeatureCollection(features)
    with open(Jsonfilename, 'w') as outfile:
        geojson.dump(feature_collection, outfile)
dropoff_by_neighborhood(collection, 'dropoff_by_neighborhood.geojson')

'Calculated Totals'
11.576557325215003
